## <span style="color:#ff5f27">👨🏻‍🏫 Train Ranking Model </span>

In this notebook, we will train a ranking model using gradient boosted trees. 

In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, precision_recall_fscore_support
import joblib

In [2]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

2025-05-26 20:20:04,056 INFO: Initializing external client
2025-05-26 20:20:04,062 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-26 20:20:05,529 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


### get feature groups

In [23]:

users_fg = fs.get_feature_group(
    name="users",
    version=1
)

events_fg = fs.get_feature_group(
    name="events",
    version=1
)

weather_rank_fg = fs.get_feature_group(
    name="weather_ranking",
    version=1
)

no_weather_rank_fg = fs.get_feature_group(
    name="no_weather_ranking",
    version=1
)

## <span style="color:#ff5f27">⚙️ Feature View Creation </span>

In [24]:
# Select features
selected_features_customers = users_fg.select_all()

fs.get_or_create_feature_view( 
    name='users',
    query=selected_features_customers,
    version=1,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/users/version/1


In [25]:
# Select features
selected_features_articles = events_fg.select_all()

fs.get_or_create_feature_view(
    name='events',
    query=selected_features_articles,
    version=1,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/events/version/1


In [9]:
NO_WEATHER_SELECTED_FEATURES =['interaction_distance_to_event', 'event_type', 'event_city',
       'attendance_rate', 'event_indoor_capability', 'user_city', 'age',
       'user_interests','interaction_label']

WEATHER_SELECTED_FEATURES =['interaction_distance_to_event', 'event_type', 'event_city', 
       'weather_condition', 'temperature', 'attendance_rate',
       'event_indoor_capability', 'user_city', 'indoor_outdoor_preference',
       'age', 'user_interests','interaction_label']


In [30]:
# Select weather features
features_weather_ranking = weather_rank_fg.select(WEATHER_SELECTED_FEATURES)
# Select no weather features
features_no_weather_ranking = no_weather_rank_fg.select(NO_WEATHER_SELECTED_FEATURES)

In [32]:
# Create feature view for weather ranking
feature_view_ranking_weather = fs.get_or_create_feature_view(
    name='weather_ranking',
    query=features_weather_ranking,
    labels=['interaction_label'],
    version=1,
)
# Create feature view for no weather ranking
feature_view_ranking_no_weather = fs.get_or_create_feature_view(
    name='no_weather_ranking',
    query=features_no_weather_ranking,
    labels=["interaction_label"],
    version=1,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/weather_ranking/version/1
Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/no_weather_ranking/version/1


---

---

---

---

## <span style="color:#ff5f27">🗄️ Train Data loading </span>

In [5]:
# Get feature views weather ranking
feature_view_ranking_weather = fs.get_feature_view(name='weather_ranking', version=1)


In [6]:
# Get feature views no weather ranking
feature_view_ranking_no_weather = fs.get_feature_view(name='no_weather_ranking', version=1)


In [7]:
# NO_WEATHER_SELECTED_FEATURES =['interaction_distance_to_event', 'event_type', 'event_city',
#        'attendance_rate', 'event_indoor_capability', 'user_city', 'age',
#        'user_interests']

# WEATHER_SELECTED_FEATURES =['interaction_distance_to_event', 'event_type', 'event_city', 
#        'weather_condition', 'temperature', 'attendance_rate',
#        'event_indoor_capability', 'user_city', 'indoor_outdoor_preference',
#        'age', 'user_interests']


In [ ]:
# from sklearn.model_selection import train_test_split
# users_df = pd.read_csv('/home/nkama/masters_thesis_project/thesis/notebooks/users.csv')
# events_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/notebooks/events.csv")
# interactions_df = pd.read_csv('/home/nkama/masters_thesis_project/thesis/notebooks/interactions.csv')

# merged_df = pd.merge(interactions_df, users_df, on='user_id')
# merged_df = pd.merge(merged_df, events_df, on='event_id') 


# X_weather = merged_df[WEATHER_SELECTED_FEATURES]
# y_weather = merged_df["interaction_label"]

# X_no_weather = merged_df[NO_WEATHER_SELECTED_FEATURES]
# y_no_weather = merged_df["interaction_label"]

# # Split merged_df into train and test using train_test_split
# weather_X_train, weather_X_val, weather_y_train, weather_y_val = train_test_split(X_weather,y_weather, test_size=0.2, random_state=42)
# no_weather_X_train, no_weather_X_val, no_weather_y_train, no_weather_y_val = train_test_split(X_no_weather,y_no_weather, test_size=0.2, random_state=42)
# X_weather.head()

In [11]:
# Get training and validation data directly from feature views for weather ranking
weather_X_train, weather_X_val, weather_y_train, weather_y_val = \
    feature_view_ranking_weather.train_test_split(
    test_size=0.1,
    description='Weather ranking training dataset',
)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.83s) 
2025-05-26 20:22:19,465 WARNING: VersionWarning: Incremented version to `2`.



In [12]:
weather_X_train.head()

,interaction_distance_to_event,event_type,event_city,weather_condition,temperature,attendance_rate,event_indoor_capability,user_city,indoor_outdoor_preference,age,user_interests
0,16.008455,Music & Concerts,Berlin,Snow,9.7,44.756877,False,Berlin,indoor,37,tech food fashion
1,26.486551,Education & Learning,Dubai,Rain,30.3,9.121153,True,Dubai,indoor,31,fitness tech sports literature
2,56.109620,Arts & Culture,Tokyo,Windy,20.4,8.316649,True,Tokyo,indoor,22,music sports cinema travel
3,54.041743,Community & Causes,Mumbai,Clear,30.9,33.194238,False,Mumbai,any,46,literature fitness art
4,72.876835,Education & Learning,Berlin,Clear,18.4,15.177108,True,Berlin,outdoor,39,sports music literature


In [13]:
# Get training and validation data directly from feature views for no weather ranking
no_weather_X_train, no_weather_X_val, no_weather_y_train, no_weather_y_val = \
    feature_view_ranking_no_weather.train_test_split(
    test_size=0.1,
    description='No-weather ranking training dataset',
)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.21s) 
2025-05-26 20:22:42,588 WARNING: VersionWarning: Incremented version to `2`.



In [36]:
weather_X_train.columns


Index(['interaction_distance_to_event', 'event_type', 'event_city',
       'weather_condition', 'temperature', 'attendance_rate',
       'event_indoor_capability', 'user_city', 'indoor_outdoor_preference',
       'age', 'user_interests'],
      dtype='object')

In [2]:
# import pandas as pd
# users_df = pd.read_csv('/home/nkama/masters_thesis_project/thesis/partially_synthetic/data/main_data/users.csv')
# events_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/partially_synthetic/data/main_data/events.csv")
# interactions_df = pd.read_csv('/home/nkama/masters_thesis_project/thesis/partially_synthetic/data/main_data/interactions.csv')



# # Merge user/event features into interactions
# interactions_df = interactions_df.merge(users_df, on="user_id")
# interactions_df = interactions_df.merge(events_df, on="event_id", suffixes=('_user', '_event'))
# from sklearn.model_selection import train_test_split

# NO_WEATHER_SELECTED_FEATURES =['interaction_type',
#        'distance_to_event', 'interaction_label',
#         'gender', 'joinedAt', 'location', 'age',
#       'indoor_outdoor_preference', 'user_interests', 
#        'start_time', 'city', 'yes_count',
#        'maybe_count', 'invited_count', 'no_count', 'total_users', 'category', 
#        'title', 'event_type','event_indoor_capability']

# WEATHER_SELECTED_FEATURES =['interaction_type',
#        'distance_to_event', 'interaction_label',
#         'gender', 'joinedAt', 'location', 'age',
#       'indoor_outdoor_preference',
#        'start_time', 'city', 'yes_count',
#        'maybe_count', 'invited_count', 'no_count', 'total_users',
#        'weather_description', 'category','event_type',
#        'event_indoor_capability', 'temperature_2m_mean', 'precipitation_sum']

# # )
# # Splitting the dataset into features and labels
# weather_X = interactions_df[WEATHER_SELECTED_FEATURES]  # Features
# weather_y = interactions_df['interaction_label']   

# no_weather_X = interactions_df[NO_WEATHER_SELECTED_FEATURES]  # Features
# no_weather_y = interactions_df['interaction_label']                   # Labels

# # Splitting the dataset into training and evaluation sets
# weather_X_train, weather_X_val, weather_y_train, weather_y_val = \
#     train_test_split(weather_X, weather_y, test_size=0.2, random_state=42)

# no_weather_X_train, no_weather_X_val, no_weather_y_train, no_weather_y_val = \
#     train_test_split(no_weather_X, no_weather_y, test_size=0.2, random_state=42)

In [16]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import numpy as np

# Final version without text fields (title, user_interests)
def train_catboost_without_text_fields(
    train_df, val_df, train_y, val_y
):
    # Drop the text fields if present
    text_columns = ["title", "user_interests"]
    train_df = train_df.drop(columns=[col for col in text_columns if col in train_df.columns])
    val_df = val_df.drop(columns=[col for col in text_columns if col in val_df.columns])

    # Identify categorical features
    cat_features = train_df.select_dtypes(include=["object", "bool"]).columns.tolist()

    # Create CatBoost Pools
    train_pool = Pool(train_df, train_y, cat_features=cat_features)
    val_pool = Pool(val_df, val_y, cat_features=cat_features)
    # Calculate class weights
    pos_weight = len(train_y[train_y == 0]) / len(train_y[train_y == 1])


    # Train the model
    model = CatBoostClassifier(
        learning_rate=0.2,
        iterations=100,
        depth=10,
        early_stopping_rounds=5,
        use_best_model=True,
        scale_pos_weight=None,  # Handle class imbalance
        verbose=False
    )


    model.fit(train_pool, eval_set=val_pool)

    # Evaluation
    preds = model.predict(val_pool)
    precision, recall, fscore, _ = precision_recall_fscore_support(val_y, preds, average="binary")
    print("\nClassification Report:")
    print(classification_report(val_y, preds))

    metrics = {
        "precision": precision,
        "recall": recall,
        "fscore": fscore,
    }
    
    preds = model.scores = model.predict_proba(val_pool)[:, 1] 
    print("Predicted Class Distribution:", np.unique(preds, return_counts=True))

    # print("\nConfusion Matrix:")
    # print(confusion_matrix(val_y, preds))

    return model, metrics, val_pool

"CatBoost training function excluding title and user_interests."


'CatBoost training function excluding title and user_interests.'

In [39]:
weather_y_val.value_counts()

interaction_label
1                    35423
0                    14317
Name: count, dtype: int64

In [17]:
# Use this function to train on your weather / no-weather datasets
weather_model, weather_metrics, weather_val_pool = train_catboost_without_text_fields(
    train_df=weather_X_train,
    val_df=weather_X_val,
    train_y=weather_y_train,
    val_y=weather_y_val
)

#Save the models using Joblib
joblib.dump(weather_model, '/home/nkama/masters_thesis_project/thesis/models/weather_ranking_model.pkl')
print("\nModels saved successfully!")




Classification Report:
2025-05-26 20:23:16,528 WARNING: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

2025-05-26 20:23:16,590 WARNING: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

2025-05-26 20:23:16,641 WARNING: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     14317
           1       0.71      1.00      0.83     35423

    accuracy                           0.71     49740
   macro avg       0.36      0.50      0.42     49740
weighted avg       0.51      0.71      0.59     49740

Predicted Class Dis

In [18]:
feat_to_score = {
    feature: score 
    for feature, score 
    in zip(
        weather_X_train.columns, 
        weather_model.feature_importances_,
    )
}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True,
    )
)
feat_to_score

{'interaction_distance_to_event': 78.81059182870735,
 'temperature': 5.4812235988230995,
 'attendance_rate': 5.267535880721474,
 'age': 4.458258420067622,
 'event_indoor_capability': 4.368699378748474,
 'indoor_outdoor_preference': 1.6136908929319993,
 'event_type': 0.0,
 'event_city': 0.0,
 'weather_condition': 0.0,
 'user_city': 0.0}

In [19]:

# Use this function to train on your weather / no-weather datasets
no_weather_model, no_weather_metrics, no_weather_val_pool = train_catboost_without_text_fields(
    train_df=no_weather_X_train,
    val_df=no_weather_X_val,
    train_y=no_weather_y_train,
    val_y=no_weather_y_val
)

joblib.dump(no_weather_model, '/home/nkama/masters_thesis_project/thesis/models/no_weather_ranking_model.pkl')
print("\nModels saved successfully!")


Classification Report:
2025-05-26 20:23:42,399 WARNING: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

2025-05-26 20:23:42,475 WARNING: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

2025-05-26 20:23:42,522 WARNING: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     14202
           1       0.71      1.00      0.83     35538

    accuracy                           0.71     49740
   macro avg       0.36      0.50      0.42     49740
weighted avg       0.51      0.71      0.60     49740

Predicted Class Dis

In [20]:

feat_to_score = {
    feature: score 
    for feature, score 
    in zip(
        no_weather_X_train.columns, 
        no_weather_model.feature_importances_,
    )
}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True,
    )
)
feat_to_score

{'interaction_distance_to_event': 81.09057497466972,
 'attendance_rate': 6.350488537822829,
 'age': 5.325933405187695,
 'event_indoor_capability': 3.565567447048702,
 'user_city': 3.2533100172012923,
 'event_type': 0.25709789201841254,
 'event_city': 0.15702772605133564}

In [8]:
# from sklearn.metrics import roc_auc_score, average_precision_score, ndcg_score, precision_score, recall_score
# import numpy as np


# def evaluate_ranking_model_proba(model, val_pool, val_y, k_list=[5, 10]):
#     """
#     Evaluate a CatBoost ranking model using predicted probabilities, not binary class outputs.
#     """

#     # Predict class probabilities (not class labels)
#     proba = model.predict_proba(val_pool)[:, 1]  # Probability for class 1

#     results = {
#         "AUC": roc_auc_score(val_y, proba),
#         "Average Precision (MAP)": average_precision_score(val_y, proba),
#     }

#     # Convert to numpy arrays
#     true_labels = np.array(val_y)
#     predicted_scores = np.array(proba)

#     # Sort by predicted score
#     sorted_indices = np.argsort(predicted_scores)[::-1]
#     sorted_true = true_labels[sorted_indices]

#     for k in k_list:
#         top_k = sorted_true[:k]
#         precision_at_k = np.mean(top_k)
#         recall_at_k = np.sum(top_k) / np.sum(true_labels)
#         ndcg_at_k = ndcg_score(
#             y_true=true_labels.reshape(1, -1),
#             y_score=predicted_scores.reshape(1, -1),
#             k=k
#         )



#         results[f"Precision@{k}"] = precision_at_k
#         results[f"Recall@{k}"] = recall_at_k
#         results[f"NDCG@{k}"] = ndcg_at_k

#     return results

# "✅ Evaluation function ready: scores ranking model using AUC, MAP, Precision@K, Recall@K, and NDCG@K."


'✅ Evaluation function ready: scores ranking model using AUC, MAP, Precision@K, Recall@K, and NDCG@K.'

In [ ]:

# # Evaluate weather-aware model
# weather_scores = evaluate_ranking_model_proba(
#     model=weather_model,
#     val_pool=weather_val_pool,
#     val_y=weather_y_val
# )

# # Evaluate no-weather model
# no_weather_scores = evaluate_ranking_model_proba(
#     model=no_weather_model,
#     val_pool=no_weather_val_pool,
#     val_y=no_weather_y_val
# )

# # Compare results
# print("Weather Model Scores:")
# for k, v in weather_scores.items():
#     print(f"{k}: {v:.4f}")

# print("\nNo-Weather Model Scores:")
# for k, v in no_weather_scores.items():
#     print(f"{k}: {v:.4f}")


In [3]:
# Connect to Hopsworks Model Registry
mr = project.get_model_registry()

In [21]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Create model schema for weather ranking model
input_example = weather_X_train.sample().to_dict("records")
input_schema = Schema(weather_X_train)
output_schema = Schema(weather_y_train)
model_schema = ModelSchema(input_schema, output_schema)

weather_ranking_model = mr.python.create_model(
    name="weather_ranking_model", 
    metrics=weather_metrics,
    model_schema=model_schema,
    input_example=input_example,
    description="Ranking model that scores item candidates",
)
weather_ranking_model.save("/home/nkama/masters_thesis_project/thesis/models/weather_ranking_model.pkl")

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /home/nkama/masters_thesis_project/thesis/models/weather_ranking_model.pkl: 0.000%|          | 0/674…

Uploading /home/nkama/masters_thesis_project/thesis/notebooks/input_example.json: 0.000%|          | 0/354 ela…

Uploading /home/nkama/masters_thesis_project/thesis/notebooks/model_schema.json: 0.000%|          | 0/1039 ela…

Model created, explore it at https://c.app.hopsworks.ai:443/p/1220788/models/weather_ranking_model/2


Model(name: 'weather_ranking_model', version: 2)

In [22]:
# Create model schema for no weather ranking model  
input_example = no_weather_X_train.sample().to_dict("records")
input_schema = Schema(no_weather_X_train)
output_schema = Schema(no_weather_y_train)
model_schema = ModelSchema(input_schema, output_schema)

no_weather_ranking_model = mr.python.create_model(
    name="no_weather_ranking_model", 
    metrics=no_weather_metrics,
    model_schema=model_schema,
    input_example=input_example,
    description="Ranking model that scores item candidates",
)
no_weather_ranking_model.save("/home/nkama/masters_thesis_project/thesis/models/no_weather_ranking_model.pkl")

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /home/nkama/masters_thesis_project/thesis/models/no_weather_ranking_model.pkl: 0.000%|          | 0/…

Uploading /home/nkama/masters_thesis_project/thesis/notebooks/input_example.json: 0.000%|          | 0/249 ela…

Uploading /home/nkama/masters_thesis_project/thesis/notebooks/model_schema.json: 0.000%|          | 0/799 elap…

Model created, explore it at https://c.app.hopsworks.ai:443/p/1220788/models/no_weather_ranking_model/2


Model(name: 'no_weather_ranking_model', version: 2)